In [ ]:
# 6-9 파이썬 언어에서 다음에 대해 조사하고 예제코드를 실행하여 보라
# 1. 파이썬으로 싱글톤 패턴을 구현하는 방법들

# Mixin 클래스를 이용한 싱글톤 패턴 구현 예제
class SingletonMixin:
    _instances = {}

    @classmethod
    def instance(cls, *args, **kwargs):
        if cls not in cls._instances:
            print(f"{cls.__name__} 객체 새로 생성")
            cls._instances[cls] = cls(*args, **kwargs)
        else:
            print(f"{cls.__name__} 기존 객체 반환")
        return cls._instances[cls]


class BaseClass:
    pass

class MyClass(BaseClass, SingletonMixin):
    pass

class OtherClass(BaseClass, SingletonMixin):
    pass


# 실행 예시
a1 = MyClass.instance()
a2 = MyClass.instance()
b1 = OtherClass.instance()
b2 = OtherClass.instance()

print(a1 is a2)  
print(b1 is b2)  
print(a1 is b1)

# __new__ 메서드를 이용한 싱글톤 패턴 구현 예제
class Singleton(object):
    _instance = None
    def __new__(class_, *args, **kwargs):
        if not isinstance(class_._instance, class_):
            class_._instance = super().__new__(class_)
        return class_._instance


class BaseClass:
    pass

class MyClass(Singleton, BaseClass):
    pass

class OtherClass(Singleton):
    pass

a1 = MyClass()
a2 = MyClass()
b1 = OtherClass()
b2 = OtherClass()

print(a1 is a2)  
print(b1 is b2)  
print(a1 is b1)  


# 2. 파이썬에서 멀티쓰레드 구현하는 기법
from threading import Thread

def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total += i
    result.append(total)
    return

if __name__ == "__main__":
    START, END = 0, 100000000
    result = list()
    th1 = Thread(target=work, args=(1, START, END, result))
    
    th1.start()
    th1.join()

print(f"Result: {sum(result)}")
# 3. 멀티쓰레드에서 공유 변수에 대한 Critical Section 을 보호하는 기법
import threading
import time

rlock = threading.RLock()
cond = threading.Condition(rlock)
queue = []

def consumer(name):
    with cond:
        while not queue:
            print(f"[{name}] 큐 비어 있음 → wait() 진입")
            cond.wait()  # 여기서 잠듦, producer가 notify할 때까지
        item = queue.pop(0)
        print(f"[{name}] 아이템 소비: {item}")

def producer(item):
    with cond:
        print(f"[Producer] 아이템 생산: {item}")
        queue.append(item)
        cond.notify()  # 대기 중인 소비자 하나 깨움


# 실행
t1 = threading.Thread(target=consumer, args=("C1",))
t2 = threading.Thread(target=consumer, args=("C2",))

t1.start()
t2.start()

time.sleep(1)  # 소비자들이 먼저 기다리도록 잠깐 대기

producer("apple")
time.sleep(1)
producer("banana")

t1.join()
t2.join()




MyClass 객체 새로 생성
MyClass 기존 객체 반환
OtherClass 객체 새로 생성
OtherClass 기존 객체 반환
True
True
False
True
True
False
Result: 4999999950000000
[C1] 큐 비어 있음 → wait() 진입
[C2] 큐 비어 있음 → wait() 진입
[Producer] 아이템 생산: apple
[C1] 아이템 소비: apple
[Producer] 아이템 생산: banana
[C2] 아이템 소비: banana


In [24]:
# 6-10 동물(Animal)을 울음을 추상 메소드로 하는 추상클래스로 정의하고,
# 개(Dog)와 고양이(Cat)를 구체적인 클래스로 정의한 후,
# 각 동물의 울음소리를 출력하는 프로그램을 작성하시오.
# 각 동물객체는 자신의 이름을 속성으로 가지며,
# 생성자에서 이름을 초기화합니다.
# 추가적으로 Animal 클래스는 현재 생성된 동물 객체의 수를 클래스 변수로 가지고,
# 정적 메소드로 현재 동물 객체의 수를 반환하는 메소드를 구현
# 객체가 생성(__init__)될 때 객체의 수를 증가하고 삭제(__del__(self) ) 될 때 객체의 수를 감소

from abc import ABC, abstractmethod

class Animal(ABC):
    _count = 0  # 클래스 변수: 현재 생성된 동물 객체 수

    @staticmethod
    def get_count():
        return Animal._count  # 정적 메서드: 현재 동물 수 반환

    def __init__(self, name: str):
        self.name = name               # 각 동물 객체는 이름 속성 보유
        Animal._count += 1             # 객체 생성 시 개수 증가

    def __del__(self):
        # 주의: __del__ 호출 시점은 인터프리터/GC에 따라 달라질 수 있음
        Animal._count -= 1             # 객체 소멸 시 개수 감소

    @abstractmethod
    def sound(self) -> str:
        """각 동물의 울음소리를 문자열로 반환"""
        pass


class Dog(Animal):
    def sound(self) -> str:
        return "멍멍"


class Cat(Animal):
    def sound(self) -> str:
        return "야옹"


# === 사용 예시 ===
d = Dog("뽀삐")
c = Cat("나비")

# 각 동물의 울음소리 출력 (이름 포함)
print(f"{d.name}: {d.sound()}")  # 뽀삐: 멍멍
print(f"{c.name}: {c.sound()}")  # 나비: 야옹

# 현재 동물 객체 수
print("현재 동물 수:", Animal.get_count())




뽀삐: 멍멍
나비: 야옹
현재 동물 수: 0
